In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
import time
from datetime import datetime
import numpy as np

In [22]:
data = pd.read_csv("../mydata/cleandata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)

In [25]:
foods = sorted(list(set(data['product'])))
noaccfoods=[]
for i in foods:
        noaccfoods.append(i.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u") )
foods = noaccfoods

In [30]:
foods

['aceite de girasol',
 'aceite de oliva',
 'ajo',
 'albaricoque',
 'arroz',
 'azucar',
 'berenjena',
 'calabacin',
 'cebada',
 'cebolla',
 'cerdo',
 'cereza',
 'champiñon',
 'ciruela',
 'clementina',
 'coliflor',
 'cordero lechal y recental (menos de 13kg)',
 'cordero pascual (mas de 13kg)',
 'esparrago',
 'fresa',
 'harina de pipas de girasol',
 'harina de trigo',
 'harina de trigo candeal',
 'judias',
 'leche entera',
 'leche semidesnatada en polvo',
 'lechuga',
 'limon',
 'malta',
 'mandarina comun',
 'mandarina satsuma',
 'mantequilla',
 'manzana',
 'maiz',
 'melocoton',
 'melon',
 'naranja',
 'nectarina',
 'pepino',
 'pera',
 'pimiento',
 'pipas de girasol',
 'puerro',
 'queso edam',
 'queso emmental',
 'sandia',
 'suero de leche',
 'ternera',
 'tomate',
 'uva',
 'vino',
 'zanahoria']

In [3]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
# opciones.add_argument('--incognito')     

In [40]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "eroski"
prices = []
prod = []
sup = []
links = []
names = []
newprice = []
frame = {}
for f in foods:
    url = f"https://supermercado.eroski.es/es/search/results/?q={f}&suggestionsFilter=false"
    driver.get(url)
    time.sleep(1)
    try: #cookies
        driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
    except:
        pass
    time.sleep(.7)
    for i in range(1,15):
        try:
            prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(4) > div:nth-child(2) > p:nth-child(1) > span:nth-child(2)').text)
        except:
            try:
                prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i+1}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(3) > div:nth-child(2) > span:nth-child(2) > span:nth-child(2)').text)
            except:
                prices.append(np.nan)
        try:
            names.append(driver.find_element_by_css_selector(f'#productListZone > div:nth-child({i+1}) > div > div.product-description > div.description-text > h2:nth-child(3) > a').text)
        except:
            names.append(np.nan)
        prod.append(f)
        sup.append(supermarket)
        links.append(url)
#         try:
#             prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(4) > div:nth-child(2) > p:nth-child(1) > span:nth-child(2)').text)
#             names.append(driver.find_element_by_css_selector(f'#productListZone > div:nth-child({i+1}) > div > div.product-description > div.description-text > h2:nth-child(3) > a').text)
#             prod.append(f)
#             sup.append(supermarket)
#             links.append(url)
    
#         except:
#             try:
#                 prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i+1}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(3) > div:nth-child(2) > span:nth-child(2) > span:nth-child(2)').text)
#                 names.append(driver.find_element_by_css_selector(f'#productListZone > div:nth-child({i+1}) > div > div.product-description > div.description-text > h2:nth-child(3) > a').text)
#                 prod.append(f)
#                 sup.append(supermarket)
#                 links.append(url)
#             except:
#                 pass
frame['price'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['link'] = links
frame['name'] = names

scrap = pd.DataFrame(frame)
for i, row in scrap.iterrows():
    if row['price'] == '':
        newprice.append(np.nan)
    else:
        newprice.append(row['price'])
scrap['price'] = newprice
scrap.dropna(how='any',inplace=True)
scrap['price'] = [float((i.replace(",",".").replace("€","").strip())) for i in scrap['price']]
scrap['name'] = [(i.replace("%","por ciento").replace('"','').strip()) for i in scrap['name']]

if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
now = str(datetime.now())[:19].replace(":","_")
scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv', index=False)
scrap.to_csv(f'../mydata/scraps_{supermarket}/{supermarket}.csv', index=False)

/tmp/ipykernel_108/4252199312.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_108/4252199312.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_108/4252199312.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(4) > div:nth-child(2) > p:nth-child(1) > span:nth-child(2)').text)
/tmp/ipykernel_108/4252199312.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'div.product-item-lineal:nth-child({i+1}) > div:nth-chi